# Using function on scrap.ipynb

Importing libraries


In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3

session = requests.Session()

Details of every products(mobiles and laptops)

In [17]:
def extract_product_details(link, category):
    try:
        r = session.get(link)
        soup = BeautifulSoup(r.text, 'lxml')

        tag = soup.find('div', class_='off-ribbon-inner').text.strip() if soup.find('div', class_='off-ribbon-inner') else 'No off'
        name = soup.find('h1').text.strip() if soup.find('h1') else 'None'
        # feature = soup.find('div', class_='main-feature').text.strip() if soup.find('div', class_='main-feature') else None
        oldprice = (soup.find('strike').text.strip().replace('Rs.','').replace('Rs','').replace(' ','')) if soup.find('strike') else 0
        newprice = int(soup.find('input',{'name':'price'})['value']) if soup.find('input', {'name':'price'}) else   0
        imgurl = soup.find('img', class_='simpleLens-big-image')['src'] if soup.find('img', class_='simpleLens-big-image') else 'None'
        
        product = {
            'Tag': tag,
            'Name': name,
            # 'feature': feature,
            'Oldprice': oldprice,
            'Newprice': newprice,
            'Category': category,
            'Imgurl': imgurl
        }
        return product
    except :
        print(f"Error extracting details for {link}")
        return None

Links of every products

In [18]:
def get_product_links(url, category):
    product_links = []
    try:
        r = session.get(url)
        soup = BeautifulSoup(r.text, 'lxml')
        box = soup.find_all('div', class_="col-sm-4 col-md-3")

        for link in box:
            href = link.find('a')['href']
            product_links.append(href)

        return product_links
    except :
        print(f"Error retrieving product links from {url}")
        return []

List to store all products

In [19]:
all_products = []

To scrape laptops

In [20]:
laptop_links = get_product_links("https://wholesale.com.np/category/index/laptops-computers?subcat=LAPTOPS", "laptop")

for link in laptop_links:
    laptop = extract_product_details(link, "laptop")
    
    all_products.append(laptop)

To scrape mobiles

In [21]:
moblinks = []
for x in range(1, 3):
    mob_links = get_product_links(f'https://wholesale.com.np/category/index/mobile-phone-tablets?subcat=MOBILE%20PHONE&page={x}', "mobile")
    
    moblinks.extend(mob_links)

for link in moblinks:
    phone = extract_product_details(link, "mobile")
   
    all_products.append(phone)

Creating dataframe 

In [22]:
df = pd.DataFrame(all_products)

# Database

In [23]:
conn=sqlite3.connect("scrapped.db")
cursor=conn.cursor()
# cursor.execute('''DELETE FROM products''')
cursor.execute('''CREATE TABLE IF NOT EXISTS products(Tag TEXT,Name TEXT,Oldprice INTEGER ,Newprice INTEGER,Category TEXT,Imgurl TEXT)''')

df.to_sql('products', conn, if_exists='append', index=False)

conn.commit()

df = pd.read_sql_query("SELECT * FROM products ", conn)
print(df.replace(r'\n', ' ', regex=True))
conn.close()

         Tag                                               Name  Oldprice  \
0    2 % off               HP 240 G4 I5-6200/ 4GB RAM (V3F08PA)     57000   
1    2 % off                   DELL INSPIRON 15 5559-I5-4GB RAM     63990   
2    2 % off                       DELL INSPIRON 15 5559-I5-4GB     59990   
3    2 % off                  DELL INPSIRATION 3459 -I5-4GB RAM     54490   
4    2 % off      DELL INSPIRON 3558-I3-4GB RAM-500GB HARD DISK     41500   
5    2 % off                              DELL INSPIRON 3458-I3     41490   
6    2 % off  DELL INSPIRON 11 3147   CELRON WITH TOUCH DISPLAY     41990   
7    3 % off    DELL INSPIRON 15 3551 / 2 GB RAM / 15.6 DISPLAY     36850   
8    1 % off  DELL INSPIRON 15 5559 6TH GEN CI7 8GB 1TB 4GB ...     81400   
9    1 % off                                ACER ASPIRE V13 371     67900   
10   3 % off                                       ASUS XMA 200     39405   
11   1 % off                                             HP 242     78750   

# Saving as CSV

In [24]:
df.to_csv('scrapped_products.csv',index=False)